In [2]:
from flask import Flask, request, jsonify, send_file
import pyodbc

app = Flask(__name__)

def get_db_connection():
    SERVER = r'DESKTOP-5NV5SJ3\SQLEXPRESS'
    DATABASE = r'nithiy'
    conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';Trusted_Connection=yes;')
    return conn

def get_bot_response(user_message):
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        keyword_pattern = f'%{user_message}%'
        cursor.execute("SELECT Response FROM Responses WHERE Keyword LIKE ?", (keyword_pattern,))
        row = cursor.fetchone()
        return row.Response if row else "I didn't understand that."
    except pyodbc.Error as e:
        print("Database error:", e)
        return "An error occurred."
    finally:
        conn.close()



@app.route('/')
def index():
    return send_file(r'C:\Users\Converse\OneDrive - CONVERSE Data Solutions\mini project\index.html')

@app.route('/chat', methods=['POST'])
def chat():
    user_message = request.form['message']
    bot_response = get_bot_response(user_message)

    # Save the conversation to the database
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('INSERT INTO ChatHistory (UserMessage, BotResponse) VALUES (?, ?)', (user_message, bot_response))
    conn.commit()
    conn.close()

    return jsonify({'user_message': user_message, 'bot_response': bot_response})

if __name__ == '__main__':
    app.run(debug=True,port=5003,use_reloader=False)  #


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5003
Press CTRL+C to quit
127.0.0.1 - - [22/Aug/2024 17:43:08] "GET / HTTP/1.1" 304 -
127.0.0.1 - - [22/Aug/2024 17:43:09] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [22/Aug/2024 17:43:12] "POST /chat HTTP/1.1" 200 -
127.0.0.1 - - [22/Aug/2024 17:43:19] "POST /chat HTTP/1.1" 200 -
127.0.0.1 - - [22/Aug/2024 17:44:22] "POST /chat HTTP/1.1" 200 -
